In [26]:
from mlcomp.data.load import load_regression_train, load_regression_test
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.metrics import mean_squared_error
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

In [27]:
df = load_regression_train()

# remove everything before 01-01-2014
df = df.loc[df["timestamp"] > 1388530800 * 1000]

# remove everything before 01-01-2011
# df = df.loc[df['timestamp'] > 1262300400 * 1000]

df = df.drop_duplicates()



X = df.drop(columns="rating")
y = df["rating"]

# print(y.value_counts())

# rus = RandomUnderSampler(random_state=42)
# X, y = rus.fit_resample(X, y)

# print(y.value_counts())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [28]:
clf = XGBRFRegressor()
clf.fit(X_train, y_train)

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               objective='reg:squarederror', predictor=None, random_state=None,
               reg_alpha=None, ...)

In [29]:
pred = clf.predict(X_test)

# squared=False <=> RMSE
mean_squared_error(y_test, pred, squared=False)

1.083670712974861

In [30]:
test = load_regression_test()

p = clf.predict(test)

df = pd.DataFrame(p, columns=["Predicted"])
df.to_csv('pred.csv', index_label="Id")

### Model Doc:


#### CatBoost
##### Config 1:
- 90/10 TTS
- Drop before 2014
- Default settings
- Val RMSE: 1.0709950168363778
- Kaggle: 1.017992

##### Config 2:
- Same as Config 1, but we undersample using imblearn's RandomUnderSampler
- Val RMSE: 1.382560782995256
- Kaggle: 1.6511


#### XGBRegressor:
##### Config 1:
- 90/10 TTS
- Drop before 2014
- Default settings
- Val RMSE: 1.048387264967563
- Kaggle: 1.00066

##### Config 2:
- 90/10 TTS
- Drop before 2011
- Default settings
- Val RMSE: 1.051376903446028
- Kaggle: 0.99867


#### XGBRFRegressor:
##### Config 1:
- 90/10 TTS
- Drop before 2011
- Default settings
- Val RMSE: 1.086789539750634
- Kaggle: 1.02778

##### Config 2:
- 90/10 TTS
- Drop before 2011
- Default settings
- Val RMSE: 1.083670712974861
- Kaggle: 0.02809